# Artwork Sequence Prediction Evaluation

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [ ]:
CONFIG_PATH = '/root/work/artwork_sequence/train_test_configuration'

In [ ]:
window_index = 4

museum_sequence_path = {
    'x_train' : os.path.join(CONFIG_PATH, 'X_train.csv'),
    'x_test' : os.path.join(CONFIG_PATH, 'X_test.csv'),
    'x_train_matrix' : os.path.join(CONFIG_PATH, 'X_train_matrix.npy'),
    'x_test_matrix' : os.path.join(CONFIG_PATH, 'X_test_matrix.npy'),
    'weights_folder' : os.path.join(CONFIG_PATH, 'config_'+str(window_index)+'/trained_model_weights')
}
museum_sequence_path

## Load data

In [ ]:
df_x_train = pd.read_csv(museum_sequence_path['x_train'], index_col=0)
df_x_test = pd.read_csv(museum_sequence_path['x_test'], index_col=0)
x_train_matrix = np.load(museum_sequence_path['x_train_matrix'])
x_test_matrix = np.load(museum_sequence_path['x_test_matrix'])
df_x_train.head()

In [ ]:
x_train_matrix.shape

In [ ]:
df_x_test['tour_path'][39]

## Reset Tensorflow session

In [ ]:
tf.keras.backend.clear_session()

## Config  data to fit with the model input

Because the **Prediction feature model** split the data into training and validation dataset, it is necessary to give all the data in only one block

**Define timeline**

In [ ]:
timeline = np.arange(x_train_matrix.shape[0] + x_test_matrix.shape[0])
timeline.shape

**Define configuration to deal with the windowed dataset**

In [ ]:
split_time = x_train_matrix.shape[0]

X = np.concatenate((x_train_matrix, x_test_matrix))

window_size = window_index

batch_size = 128
shuffle_buffer_size = 300

## Create and Load models

**Helper function to load weights**

In [ ]:
def load_weights(model, index, museum_sequence_path):
    #Find the folder where the weights are saved
    model_feature_folder = os.path.join(museum_sequence_path['weights_folder'], 'model_feature_'+str(index))    
    #Load weights
    model.load_weights(os.path.join(model_feature_folder, 'weights_feature_'+str(index)))
    return model

**Get models**

In [ ]:
from Prediction_model_feature import Prediction_model_feature

In [ ]:
n_features = X.shape[1]
models = []

In [ ]:
from IPython.display import clear_output
import time

start_time = time.time()

for i in range(n_features):
    clear_output(wait=True)
    #Create model
    model_prediction = Prediction_model_feature(
        X=X[:, 0],
        split_time=split_time,
        train_batch_size=batch_size, 
        val_batch_size=batch_size, 
        window_size=window_size, 
        shuffle_buffer=shuffle_buffer_size,
        name="feature " + str(0))
    model = model_prediction.get_model()
    #Load weights
    model =load_weights(model, i, museum_sequence_path)
    models.append(model)
    
print("--- %s seconds ---" % (time.time() - start_time))
len(models)

## Evaluate model

In [ ]:
from utils_plot import plot_series, plot_train_history, plot_prediction

In [ ]:
def model_forecast(model, series, window_size, batch_size):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.map(lambda w: (w[:]))
    ds = ds.batch(batch_size)
    forecast = model.predict(ds)
    return forecast

**Predict validation dataset**

In [ ]:
df_evaluation = pd.DataFrame({'feature' : [],
                             'forecast': [],
                             'x_valid':[],
                             'mae':[]})
df_evaluation

In [ ]:
for feature in range(n_features):
    #Predict feature i
    x_feature = X[:,feature]
    rnn_forecast = model_forecast(models[feature], x_feature, window_size, batch_size)
    rnn_forecast = rnn_forecast[split_time-window_size+1:,-1]

    #Get validation dataset 
    x_valid = x_feature[split_time:]

    #Compute MAE
    mae = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy().mean()

    df_evaluation = df_evaluation.append({'feature' : feature,
                    'forecast': rnn_forecast,
                    'x_valid':x_valid,
                    'mae':mae
                   }, 
                   ignore_index=True)

df_evaluation.head()

In [ ]:
df_evaluation.to_csv(os.path.join(CONFIG_PATH, 'config_'+str(window_size)+'/forecast.csv'), index=False)

**Plot forecast for a single feature**

In [ ]:
PLOT_PATH = '/root/work/artwork_sequence/plots'

In [ ]:
feature = 5
rnn_forecast = df_evaluation.loc[feature,'forecast']
x_valid = df_evaluation.loc[feature,'x_valid']
timeline = np.arange(rnn_forecast.shape[0])

In [ ]:
plot = plot_series(timeline, [(x_valid, 'x_valid'), (rnn_forecast, 'rnn')])
plot.title('Forescast artwork sequence')
plot.savefig(os.path.join(PLOT_PATH, 'forescast_sequence_2.png'))
plot.show()

**MAE**

In [ ]:
df_evaluation['mae'].mean()